In [3]:
! pip install pygame
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pygame
pygame.init()

# Spielparameter
screen_width = 800
screen_height = 600
play_width = 300  # meaning 300 // 10 = 30 width per block
play_height = 600  # meaning 600 // 20 = 20 height per block
block_size = 30

top_left_x = (screen_width - play_width) // 2
top_left_y = screen_height - play_height
#Farben
black = (0, 0, 0)
white = (255, 255, 255)
red = (200, 0, 0)
green = (0, 200, 0)
blue = (0, 0, 255)
purple = (150, 0, 150)
yellow = (255, 255, 0)
orange = (255, 165, 0)
pink = (255, 100, 100)


#Formen
S = [['.....',
      '.....',
      '..00.',
      '.00..',
      '.....'],
     ['.....',
      '..0..',
      '..00.',
      '...0.',
      '.....']]
Z = [['.....',
      '.....',
      '.00..',
      '..00.',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '.0...',
      '.....']]
O = [['.....',
      '.....',
      '.00..',
      '.00..',
      '.....']]

I = [['..0..',
      '..0..',
      '..0..',
      '..0..',
      '.....'],
     ['.....',
      '0000.',
      '.....',
      '.....',
      '.....']]



L = [['.....',
      '...0.',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..0..',
      '..00.',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '.0...',
      '.....'],
     ['.....',
      '.00..',
      '..0..',
      '..0..',
      '.....']]

T = [['.....',
      '..0..',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..00.',
      '..0..',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '..0..',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '..0..',
      '.....']]

shape = [S, Z, O, I, L, T]
shape_colors = [red, green, blue, purple, yellow, orange, pink]


#class Block
class Block:
    def __init__(self, x, y, shape):
        self.x = x
        self.y = y
        self.shape = shape
        self.color = shape_colors[shape]
        self.rotation = 0
  # Funktionen
def create_grid(locked_positions={}):
    grid = [[black for _ in range(10)] for _ in range(20)]
    for y in range(20):
        for x in range(10):
            if (x, y) in locked_positions:
                color = locked_positions[(x, y)]
                grid[y][x] = color
    return grid

def convert_shape_format(block):
    positions = []
    format = block.shape[block.rotation % len(block.shape)]
    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                positions.append((block.x + j, block.y + i))
    for i, pos in enumerate(positions):
        positions[i] = (pos[0] - 2, pos[1] - 4)
    return positions

def valid_space(block, grid):
    accepted_positions = [[(x, y) for x in range(10) if grid[y][x] == black] for y in range(20)]
    accepted_positions = [x for sub in accepted_positions for x in sub]
    formatted = convert_shape_format(block)
    for pos in formatted:
        if pos not in accepted_positions:
            if pos[1] > -1:
                return False
    return True

def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < 1:
            return True
    return False

def get_shape():
    return Block(5, 0, random.choice(shapes))

def draw_text_middle(text, size, color, surface):
    font = pygame.font.Font(pygame.font.get_default_font(), size, bold=False)
    label = font.render(text, 1, color)
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), top_left_y + play_height / 2 - (label.get_height() / 2)))

def draw_grid(surface, grid):
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            pygame.draw.rect(surface, grid[y][x], (top_left_x + x * block_size, top_left_y + y * block_size, block_size, block_size), 0)
    draw_grid_lines(surface)

def draw_grid_lines(surface):
    for y in range(20):
        pygame.draw.line(surface, white, (top_left_x, top_left_y + y * block_size), (top_left_x + play_width, top_left_y + y * block_size))
    for x in range(10):
        pygame.draw.line(surface, white, (top_left_x + x * block_size, top_left_y), (top_left_x + x * block_size, top_left_y + play_height))

def clear_rows(grid, locked):
    increment = 0
    for y in range(len(grid)-1, -1, -1):
        row = grid[y]
        if black not in row:
            increment += 1
            ind = y
            for x in range(len(row)):
                try:
                    del locked[(x, y)]
                except:
                    continue
    if increment > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                new_key = (x, y + increment)
                locked[new_key] = locked.pop(key)
    return increment

def draw_next_shape(shape, surface):
    font = pygame.font.Font(pygame.font.get_default_font(), 30)
    label = font.render('Next Shape', 1, white)
    start_x = top_left_x + play_width + 50
    start_y = top_left_y + play_height // 2 - 100
    format = shape.shape[shape.rotation % len(shape.shape)]
    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                pygame.draw.rect(surface, shape.color, (start_x + j * block_size, start_y + i * block_size, block_size, block_size), 0)
    surface.blit(label, (start_x + 10, start_y - 30))

def draw_window(surface, grid, score=0):
    surface.fill(black)
    font = pygame.font.Font(pygame.font.get_default_font(), 60)
    label = font.render('TETRIS', 1, white)
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), 30))
    font = pygame.font.Font(pygame.font.get_default_font(), 30)
    label = font.render(f'Score: {score}', 1, white)
    surface.blit(label, (top_left_x + play_width + 50, top_left_y + play_height // 2 - 150))
    draw_grid(surface, grid)
    draw_grid_lines(surface)

def main():
    locked_positions = {}
    grid = create_grid(locked_positions)
    change_block = False
    run = True
    current_block = get_shape()
    next_block = get_shape()
    clock = pygame.time.Clock()
    fall_time = 0
    level_time = 0
    score = 0

    while run:
        grid = create_grid(locked_positions)
        fall_speed = 0.27
        fall_time += clock.get_rawtime()
        level_time += clock.get_rawtime()
        clock.tick()

        if level_time / 1000 > 5:
            level_time = 0
            if fall_speed > 0.12:
                fall_speed -= 0.005

        if fall_time / 1000 >= fall_speed:
            fall_time = 0
            current_block.y += 1
            if not (valid_space(current_block, grid)) and current_block.y > 0:
                current_block.y -= 1
                change_block = True

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.display.quit()
                quit()

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    current_block.x -= 1
                    if not valid_space(current_block, grid):
                        current_block.x += 1

                if event.key == pygame.K_RIGHT:
                    current_block.x += 1
                    if not valid_space(current_block, grid):
                        current_block.x -= 1

                if event.key == pygame.K_DOWN:
                    current_block.y += 1
                    if not valid_space(current_block, grid):
                        current_block.y -= 1

                if event.key == pygame.K_UP:
                    current_block.rotation = current_block.rotation + 1 % len(current_block.shape)
                    if not valid_space(current_block, grid):
                        current_block.rotation = current_block.rotation - 1 % len(current_block.shape)

        shape_pos = convert_shape_format(current_block)

        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_block.color

        if change_block:
            for pos in shape_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = current_block.color
            current_block = next_block
            next_block = get_shape()
            change_block = False
            score += clear_rows(grid, locked_positions) * 10

        draw_window(win, grid, score)
        draw_next_shape(next_block, win)
        pygame.display.update()

        if check_lost(locked_positions):
            draw_text_middle("YOU LOST", 80, white, win)
            pygame.display.update()
            pygame.time.delay(2000)
            run = False

def main_menu():
    run = True
    while run:
        win.fill(black)
        draw_text_middle('Press Any Key To Play', 60, white, win)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.KEYDOWN:
                main()
    pygame.quit()

win = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Tetris')
main_menu()


pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


TypeError: function takes at most 2 arguments (3 given)

In [5]:
import pygame
import random

# Initialisiere Pygame
pygame.init()

# Spielparameter
screen_width = 800
screen_height = 600
play_width = 300  # meaning 300 // 10 = 30 width per block
play_height = 600  # meaning 600 // 20 = 20 height per block
block_size = 30

top_left_x = (screen_width - play_width) // 2
top_left_y = screen_height - play_height

# Farben
black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
yellow = (255, 255, 0)
cyan = (0, 255, 255)
purple = (128, 0, 128)
orange = (255, 165, 0)

# Formen
S = [['.....',
      '.....',
      '..00.',
      '.00..',
      '.....'],
     ['.....',
      '..0..',
      '..00.',
      '...0.',
      '.....']]

Z = [['.....',
      '.....',
      '.00..',
      '..00.',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '.0...',
      '.....']]

I = [['.....',
      '..0..',
      '..0..',
      '..0..',
      '..0..'],
     ['.....',
      '0000.',
      '.....',
      '.....',
      '.....']]

O = [['.....',
      '.....',
      '.00..',
      '.00..',
      '.....']]

J = [['.....',
      '.0...',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..00.',
      '..0..',
      '..0..',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '...0.',
      '.....'],
     ['.....',
      '..0..',
      '..0..',
      '.00..',
      '.....']]

L = [['.....',
      '...0.',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..0..',
      '..00.',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '.0...',
      '.....'],
     ['.....',
      '.00..',
      '..0..',
      '..0..',
      '.....']]

T = [['.....',
      '..0..',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..00.',
      '..0..',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '..0..',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '..0..',
      '.....']]

shapes = [S, Z, I, O, J, L, T]
shape_colors = [green, red, cyan, yellow, blue, orange, purple]

# Block-Klasse
class Block:
    def __init__(self, x, y, shape):
        self.x = x
        self.y = y
        self.shape = shape
        self.color = shape_colors[shapes.index(shape)]
        self.rotation = 0

# Funktionen
def create_grid(locked_positions={}):
    grid = [[black for _ in range(10)] for _ in range(20)]
    for y in range(20):
        for x in range(10):
            if (x, y) in locked_positions:
                color = locked_positions[(x, y)]
                grid[y][x] = color
    return grid

def convert_shape_format(block):
    positions = []
    format = block.shape[block.rotation % len(block.shape)]
    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                positions.append((block.x + j, block.y + i))
    for i, pos in enumerate(positions):
        positions[i] = (pos[0] - 2, pos[1] - 4)
    return positions

def valid_space(block, grid):
    accepted_positions = [[(x, y) for x in range(10) if grid[y][x] == black] for y in range(20)]
    accepted_positions = [x for sub in accepted_positions for x in sub]
    formatted = convert_shape_format(block)
    for pos in formatted:
        if pos not in accepted_positions:
            if pos[1] > -1:
                return False
    return True

def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < 1:
            return True
    return False

def get_shape():
    return Block(5, 0, random.choice(shapes))

def draw_text_middle(text, size, color, surface):
    font = pygame.font.Font(pygame.font.get_default_font(), size, bold=False)
    label = font.render(text, 1, color)
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), top_left_y + play_height / 2 - (label.get_height() / 2)))

def draw_grid(surface, grid):
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            pygame.draw.rect(surface, grid[y][x], (top_left_x + x * block_size, top_left_y + y * block_size, block_size, block_size), 0)
    draw_grid_lines(surface)

def draw_grid_lines(surface):
    for y in range(20):
        pygame.draw.line(surface, white, (top_left_x, top_left_y + y * block_size), (top_left_x + play_width, top_left_y + y * block_size))
    for x in range(10):
        pygame.draw.line(surface, white, (top_left_x + x * block_size, top_left_y), (top_left_x + x * block_size, top_left_y + play_height))

def clear_rows(grid, locked):
    increment = 0
    for y in range(len(grid)-1, -1, -1):
        row = grid[y]
        if black not in row:
            increment += 1
            ind = y
            for x in range(len(row)):
                try:
                    del locked[(x, y)]
                except:
                    continue
    if increment > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                new_key = (x, y + increment)
                locked[new_key] = locked.pop(key)
    return increment

def draw_next_shape(shape, surface):
    font = pygame.font.Font(pygame.font.get_default_font(), 30)
    label = font.render('Next Shape', 1, white)
    start_x = top_left_x + play_width + 50
    start_y = top_left_y + play_height // 2 - 100
    format = shape.shape[shape.rotation % len(shape.shape)]
    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                pygame.draw.rect(surface, shape.color, (start_x + j * block_size, start_y + i * block_size, block_size, block_size), 0)
    surface.blit(label, (start_x + 10, start_y - 30))

def draw_window(surface, grid, score=0):
    surface.fill(black)
    font = pygame.font.Font(pygame.font.get_default_font(), 60)
    label = font.render('TETRIS', 1, white)
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), 30))
    font = pygame.font.Font(pygame.font.get_default_font(), 30)
    label = font.render(f'Score: {score}', 1, white)
    surface.blit(label, (top_left_x + play_width + 50, top_left_y + play_height // 2 - 150))
    draw_grid(surface, grid)
    draw_grid_lines(surface)

def main():
    locked_positions = {}
    grid = create_grid(locked_positions)
    change_block = False
    run = True
    current_block = get_shape()
    next_block = get_shape()
    clock = pygame.time.Clock()
    fall_time = 0
    level_time = 0
    score = 0

    while run:
        grid = create_grid(locked_positions)
        fall_speed = 0.27
        fall_time += clock.get_rawtime()
        level_time += clock.get_rawtime()
        clock.tick()

        if level_time / 1000 > 5:
            level_time = 0
            if fall_speed > 0.12:
                fall_speed -= 0.005

        if fall_time / 1000 >= fall_speed:
            fall_time = 0
            current_block.y += 1
            if not (valid_space(current_block, grid)) and current_block.y > 0:
                current_block.y -= 1
                change_block = True

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.display.quit()
                quit()

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    current_block.x -= 1
                    if not valid_space(current_block, grid):
                        current_block.x += 1

                if event.key == pygame.K_RIGHT:
                    current_block.x += 1
                    if not valid_space(current_block, grid):
                        current_block.x -= 1

                if event.key == pygame.K_DOWN:
                    current_block.y += 1
                    if not valid_space(current_block, grid):
                        current_block.y -= 1

                if event.key == pygame.K_UP:
                    current_block.rotation = current_block.rotation + 1 % len(current_block.shape)
                    if not valid_space(current_block, grid):
                        current_block.rotation = current_block.rotation - 1 % len(current_block.shape)

        shape_pos = convert_shape_format(current_block)

        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_block.color

        if change_block:
            for pos in shape_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = current_block.color
            current_block = next_block
            next_block = get_shape()
            change_block = False
            score += clear_rows(grid, locked_positions) * 10

        draw_window(win, grid, score)
        draw_next_shape(next_block, win)
        pygame.display.update()

        if check_lost(locked_positions):
            draw_text_middle("YOU LOST", 80, white, win)
            pygame.display.update()
            pygame.time.delay(2000)
            run = False

def main_menu():
    run = True
    while run:
        win.fill(black)
        draw_text_middle('Press Any Key To Play', 60, white, win)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.KEYDOWN:
                main()
    pygame.quit()

win = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Tetris')
main_menu()


TypeError: function takes at most 2 arguments (3 given)

In [ ]:
import pygame
import random

# Initialisiere Pygame
pygame.init()

# Spielparameter
screen_width = 800
screen_height = 600
play_width = 300  # meaning 300 // 10 = 30 width per block
play_height = 600  # meaning 600 // 20 = 20 height per block
block_size = 30

top_left_x = (screen_width - play_width) // 2
top_left_y = screen_height - play_height

# Farben
black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
yellow = (255, 255, 0)
cyan = (0, 255, 255)
purple = (128, 0, 128)
orange = (255, 165, 0)

# Formen
S = [['.....',
      '.....',
      '..00.',
      '.00..',
      '.....'],
     ['.....',
      '..0..',
      '..00.',
      '...0.',
      '.....']]

Z = [['.....',
      '.....',
      '.00..',
      '..00.',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '.0...',
      '.....']]

I = [['.....',
      '..0..',
      '..0..',
      '..0..',
      '..0..'],
     ['.....',
      '0000.',
      '.....',
      '.....',
      '.....']]

O = [['.....',
      '.....',
      '.00..',
      '.00..',
      '.....']]

J = [['.....',
      '.0...',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..00.',
      '..0..',
      '..0..',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '...0.',
      '.....'],
     ['.....',
      '..0..',
      '..0..',
      '.00..',
      '.....']]

L = [['.....',
      '...0.',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..0..',
      '..00.',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '.0...',
      '.....'],
     ['.....',
      '.00..',
      '..0..',
      '..0..',
      '.....']]

T = [['.....',
      '..0..',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..00.',
      '..0..',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '..0..',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '..0..',
      '.....']]

shapes = [S, Z, I, O, J, L, T]
shape_colors = [green, red, cyan, yellow, blue, orange, purple]

# Block-Klasse
class Block:
    def __init__(self, x, y, shape):
        self.x = x
        self.y = y
        self.shape = shape
        self.color = shape_colors[shapes.index(shape)]
        self.rotation = 0

# Funktionen
def create_grid(locked_positions={}):
    grid = [[black for _ in range(10)] for _ in range(20)]
    for y in range(20):
        for x in range(10):
            if (x, y) in locked_positions:
                color = locked_positions[(x, y)]
                grid[y][x] = color
    return grid

def convert_shape_format(block):
    positions = []
    format = block.shape[block.rotation % len(block.shape)]
    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                positions.append((block.x + j, block.y + i))
    for i, pos in enumerate(positions):
        positions[i] = (pos[0] - 2, pos[1] - 4)
    return positions

def valid_space(block, grid):
    accepted_positions = [[(x, y) for x in range(10) if grid[y][x] == black] for y in range(20)]
    accepted_positions = [x for sub in accepted_positions for x in sub]
    formatted = convert_shape_format(block)
    for pos in formatted:
        if pos not in accepted_positions:
            if pos[1] > -1:
                return False
    return True

def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < 1:
            return True
    return False

def get_shape():
    return Block(5, 0, random.choice(shapes))

def draw_text_middle(text, size, color, surface):
    font = pygame.font.SysFont(pygame.font.get_default_font(), size, bold=False)
    label = font.render(text, 1, color)
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), top_left_y + play_height / 2 - (label.get_height() / 2)))

def draw_grid(surface, grid):
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            pygame.draw.rect(surface, grid[y][x], (top_left_x + x * block_size, top_left_y + y * block_size, block_size, block_size), 0)
    draw_grid_lines(surface)

def draw_grid_lines(surface):
    for y in range(20):
        pygame.draw.line(surface, white, (top_left_x, top_left_y + y * block_size), (top_left_x + play_width, top_left_y + y * block_size))
    for x in range(10):
        pygame.draw.line(surface, white, (top_left_x + x * block_size, top_left_y), (top_left_x + x * block_size, top_left_y + play_height))

def clear_rows(grid, locked):
    increment = 0
    for y in range(len(grid)-1, -1, -1):
        row = grid[y]
        if black not in row:
            increment += 1
            ind = y
            for x in range(len(row)):
                try:
                    del locked[(x, y)]
                except:
                    continue
    if increment > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                new_key = (x, y + increment)
                locked[new_key] = locked.pop(key)
    return increment

def draw_next_shape(shape, surface):
    font = pygame.font.SysFont(pygame.font.get_default_font(), 30)
    label = font.render('Next Shape', 1, white)
    start_x = top_left_x + play_width + 50
    start_y = top_left_y + play_height // 2 - 100
    format = shape.shape[shape.rotation % len(shape.shape)]
    for i, line in enumerate(format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '0':
                pygame.draw.rect(surface, shape.color, (start_x + j * block_size, start_y + i * block_size, block_size, block_size), 0)
    surface.blit(label, (start_x + 10, start_y - 30))

def draw_window(surface, grid, score=0):
    surface.fill(black)
    font = pygame.font.SysFont(pygame.font.get_default_font(), 60)
    label = font.render('TETRIS', 1, white)
    surface.blit(label, (top_left_x + play_width / 2 - (label.get_width() / 2), 30))
    font = pygame.font.SysFont(pygame.font.get_default_font(), 30)
    label = font.render(f'Score: {score}', 1, white)
    surface.blit(label, (top_left_x + play_width + 50, top_left_y + play_height // 2 - 150))
    draw_grid(surface, grid)
    draw_grid_lines(surface)

def main():
    locked_positions = {}
    grid = create_grid(locked_positions)
    change_block = False
    run = True
    current_block = get_shape()
    next_block = get_shape()
    clock = pygame.time.Clock()
    fall_time = 0
    level_time = 0
    score = 0

    while run:
        grid = create_grid(locked_positions)
        fall_speed = 0.27
        fall_time += clock.get_rawtime()
        level_time += clock.get_rawtime()
        clock.tick()

        if level_time / 1000 > 5:
            level_time = 0
            if fall_speed > 0.12:
                fall_speed -= 0.005

        if fall_time / 1000 >= fall_speed:
            fall_time = 0
            current_block.y += 1
            if not (valid_space(current_block, grid)) and current_block.y > 0:
                current_block.y -= 1
                change_block = True

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.display.quit()
                quit()

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    current_block.x -= 1
                    if not valid_space(current_block, grid):
                        current_block.x += 1

                if event.key == pygame.K_RIGHT:
                    current_block.x += 1
                    if not valid_space(current_block, grid):
                        current_block.x -= 1

                if event.key == pygame.K_DOWN:
                    current_block.y += 1
                    if not valid_space(current_block, grid):
                        current_block.y -= 1

                if event.key == pygame.K_UP:
                    current_block.rotation = current_block.rotation + 1 % len(current_block.shape)
                    if not valid_space(current_block, grid):
                        current_block.rotation = current_block.rotation - 1 % len(current_block.shape)

        shape_pos = convert_shape_format(current_block)

        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = current_block.color

        if change_block:
            for pos in shape_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = current_block.color
            current_block = next_block
            next_block = get_shape()
            change_block = False
            score += clear_rows(grid, locked_positions) * 10

        draw_window(win, grid, score)
        draw_next_shape(next_block, win)
        pygame.display.update()

        if check_lost(locked_positions):
            draw_text_middle("YOU LOST", 80, white, win)
            pygame.display.update()
            pygame.time.delay(2000)
            run = False

def main_menu():
    run = True
    while run:
        win.fill(black)
        draw_text_middle('Press Any Key To Play', 60, white, win)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.KEYDOWN:
                main()
    pygame.quit()

win = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Tetris')
main_menu()
